# 04. 音楽分析

音楽の構造や特徴を自動分析する技術を学びます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. ビート検出

In [ ]:
class BeatDetector:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        
    def onset_detection(self, audio, hop_length=512):
        """オンセット検出による簡易ビート検出"""
        # スペクトログラム計算
        f, t, Sxx = signal.spectrogram(audio, self.sample_rate, 
                                      nperseg=1024, noverlap=512)
        
        # スペクトル変化量を計算
        spectral_diff = np.diff(Sxx, axis=1)
        
        # 正の変化のみ（増加）
        spectral_diff[spectral_diff < 0] = 0
        
        # 各時刻での総変化量
        onset_strength = np.sum(spectral_diff, axis=0)
        
        # ピーク検出
        peaks, _ = signal.find_peaks(onset_strength, 
                                   height=np.mean(onset_strength) + np.std(onset_strength),
                                   distance=int(0.1 * len(onset_strength) / (t[-1] - t[0])))
        
        # 時刻に変換
        beat_times = t[1:][peaks]  # t[1:]は差分のため
        
        return beat_times, onset_strength, t[1:]
        
    def tempo_estimation(self, beat_times):
        """テンポ推定"""
        if len(beat_times) < 2:
            return 0
            
        # ビート間隔
        intervals = np.diff(beat_times)
        
        # 中央値を使用（外れ値に頑健）
        median_interval = np.median(intervals)
        
        # BPM計算
        bpm = 60.0 / median_interval
        
        return bpm

# テスト用音楽信号の生成
def create_drum_pattern(sample_rate, duration, bpm=120):
    """ドラムパターンの生成"""
    beat_interval = 60.0 / bpm
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    signal = np.zeros_like(t)
    
    # キックドラム（低周波）
    for beat_time in np.arange(0, duration, beat_interval):
        beat_idx = int(beat_time * sample_rate)
        if beat_idx < len(signal):
            # 短い低周波パルス
            pulse_duration = 0.1
            pulse_samples = int(pulse_duration * sample_rate)
            t_pulse = np.linspace(0, pulse_duration, pulse_samples)
            
            # 60Hzのサイン波にエンベロープ
            kick = np.sin(2 * np.pi * 60 * t_pulse) * np.exp(-t_pulse * 20)
            
            end_idx = min(beat_idx + pulse_samples, len(signal))
            signal[beat_idx:end_idx] += kick[:end_idx-beat_idx]
    
    # ハイハット（高周波、オフビート）
    for beat_time in np.arange(beat_interval/2, duration, beat_interval):
        beat_idx = int(beat_time * sample_rate)
        if beat_idx < len(signal):
            # 短い高周波ノイズ
            pulse_duration = 0.05
            pulse_samples = int(pulse_duration * sample_rate)
            
            np.random.seed(int(beat_time * 1000))
            hihat = np.random.normal(0, 0.3, pulse_samples) * np.exp(-np.arange(pulse_samples) * 0.001)
            
            end_idx = min(beat_idx + pulse_samples, len(signal))
            signal[beat_idx:end_idx] += hihat[:end_idx-beat_idx]
    
    return t, signal

# ビート検出のテスト
sample_rate = 22050  # 軽量化
duration = 8.0
test_bpm = 128

detector = BeatDetector(sample_rate)

# テスト信号生成
t_drum, drum_signal = create_drum_pattern(sample_rate, duration, test_bpm)

# ビート検出実行
beat_times, onset_strength, onset_times = detector.onset_detection(drum_signal)
estimated_bpm = detector.tempo_estimation(beat_times)

# 可視化
plt.figure(figsize=(16, 10))

# 元の信号
plt.subplot(3, 2, 1)
plt.plot(t_drum, drum_signal, 'b-', linewidth=1)
plt.title(f'ドラムパターン（{test_bpm} BPM）')
plt.ylabel('振幅')
plt.grid(True)

# 理論的なビート位置をマーク
theoretical_beats = np.arange(0, duration, 60.0/test_bpm)
for beat in theoretical_beats:
    plt.axvline(x=beat, color='g', linestyle='--', alpha=0.5)

# オンセット強度
plt.subplot(3, 2, 2)
plt.plot(onset_times, onset_strength, 'r-', linewidth=2)
plt.scatter(beat_times, onset_strength[np.isin(onset_times, beat_times, assume_unique=True)], 
           color='red', s=50, zorder=5)
plt.title('オンセット強度と検出されたビート')
plt.ylabel('強度')
plt.grid(True)

# スペクトログラム
plt.subplot(3, 2, 3)
f, t_spec, Sxx = signal.spectrogram(drum_signal, sample_rate, nperseg=512, noverlap=256)
plt.pcolormesh(t_spec, f, 10 * np.log10(Sxx + 1e-10), shading='gouraud', cmap='viridis')

# 検出されたビートをマーク
for beat in beat_times:
    plt.axvline(x=beat, color='red', linestyle='-', alpha=0.8, linewidth=2)

plt.title('スペクトログラムと検出ビート')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 1000)
plt.colorbar(label='振幅 (dB)')

# ビート間隔の分析
plt.subplot(3, 2, 4)
if len(beat_times) > 1:
    intervals = np.diff(beat_times)
    plt.plot(beat_times[1:], intervals, 'mo-', linewidth=2, markersize=8)
    plt.axhline(y=60.0/test_bpm, color='g', linestyle='--', 
               label=f'理論値: {60.0/test_bpm:.3f}s')
    plt.axhline(y=np.median(intervals), color='r', linestyle='-', 
               label=f'検出値: {np.median(intervals):.3f}s')
    plt.title('ビート間隔の変化')
    plt.xlabel('時間 (秒)')
    plt.ylabel('間隔 (秒)')
    plt.legend()
    plt.grid(True)

# 結果サマリー
plt.subplot(3, 2, (5, 6))
plt.axis('off')

summary_text = f"""
ビート検出結果

理論BPM: {test_bpm}
検出BPM: {estimated_bpm:.1f}
誤差: {abs(estimated_bpm - test_bpm):.1f} BPM

検出されたビート数: {len(beat_times)}
理論ビート数: {len(theoretical_beats)}

平均ビート間隔: {np.mean(np.diff(beat_times)):.3f}s
理論ビート間隔: {60.0/test_bpm:.3f}s
"""

plt.text(0.1, 0.5, summary_text, fontsize=14, verticalalignment='center',
        bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.8))

plt.tight_layout()
plt.show()

print("ビート検出の原理:")
print("1. スペクトログラムの時間変化を分析")
print("2. 急激な音響変化（オンセット）を検出")
print("3. ピーク検出でビートタイミングを特定")
print("4. ビート間隔からテンポを推定")

## 2. 音楽的特徴量

In [ ]:
class MusicFeatures:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        
    def spectral_centroid(self, audio, window_size=2048):
        """スペクトル重心（明るさの指標）"""
        hop_length = window_size // 4
        centroids = []
        
        for i in range(0, len(audio) - window_size, hop_length):
            frame = audio[i:i + window_size]
            windowed_frame = frame * np.hanning(window_size)
            
            spectrum = np.abs(fft(windowed_frame))
            freqs = fftfreq(window_size, 1/self.sample_rate)
            
            # 正の周波数のみ
            positive_freqs = freqs[:window_size//2]
            positive_spectrum = spectrum[:window_size//2]
            
            # 重心計算
            if np.sum(positive_spectrum) > 0:
                centroid = np.sum(positive_freqs * positive_spectrum) / np.sum(positive_spectrum)
                centroids.append(centroid)
            else:
                centroids.append(0)
                
        return np.array(centroids)
        
    def spectral_bandwidth(self, audio, window_size=2048):
        """スペクトル帯域幅"""
        hop_length = window_size // 4
        bandwidths = []
        centroids = self.spectral_centroid(audio, window_size)
        
        for i, centroid in enumerate(centroids):
            frame_start = i * hop_length
            frame = audio[frame_start:frame_start + window_size]
            
            if len(frame) == window_size:
                windowed_frame = frame * np.hanning(window_size)
                spectrum = np.abs(fft(windowed_frame))
                freqs = fftfreq(window_size, 1/self.sample_rate)
                
                positive_freqs = freqs[:window_size//2]
                positive_spectrum = spectrum[:window_size//2]
                
                if np.sum(positive_spectrum) > 0:
                    bandwidth = np.sqrt(np.sum(((positive_freqs - centroid) ** 2) * positive_spectrum) / 
                                      np.sum(positive_spectrum))
                    bandwidths.append(bandwidth)
                else:
                    bandwidths.append(0)
            else:
                bandwidths.append(0)
                
        return np.array(bandwidths)
        
    def zero_crossing_rate(self, audio, window_size=2048):
        """ゼロクロッシングレート"""
        hop_length = window_size // 4
        zcr = []
        
        for i in range(0, len(audio) - window_size, hop_length):
            frame = audio[i:i + window_size]
            
            # 符号変化の検出
            signs = np.sign(frame)
            sign_changes = np.diff(signs)
            zero_crossings = np.sum(sign_changes != 0)
            
            zcr.append(zero_crossings / window_size)
            
        return np.array(zcr)
        
    def rms_energy(self, audio, window_size=2048):
        """RMSエネルギー"""
        hop_length = window_size // 4
        rms = []
        
        for i in range(0, len(audio) - window_size, hop_length):
            frame = audio[i:i + window_size]
            rms_value = np.sqrt(np.mean(frame ** 2))
            rms.append(rms_value)
            
        return np.array(rms)
        
    def spectral_rolloff(self, audio, window_size=2048, percentile=85):
        """スペクトルロールオフ"""
        hop_length = window_size // 4
        rolloffs = []
        
        for i in range(0, len(audio) - window_size, hop_length):
            frame = audio[i:i + window_size]
            windowed_frame = frame * np.hanning(window_size)
            
            spectrum = np.abs(fft(windowed_frame))
            freqs = fftfreq(window_size, 1/self.sample_rate)
            
            positive_freqs = freqs[:window_size//2]
            positive_spectrum = spectrum[:window_size//2]
            
            # 累積エネルギー
            cumulative_energy = np.cumsum(positive_spectrum)
            total_energy = cumulative_energy[-1]
            
            if total_energy > 0:
                threshold = (percentile / 100.0) * total_energy
                rolloff_idx = np.where(cumulative_energy >= threshold)[0]
                
                if len(rolloff_idx) > 0:
                    rolloffs.append(positive_freqs[rolloff_idx[0]])
                else:
                    rolloffs.append(positive_freqs[-1])
            else:
                rolloffs.append(0)
                
        return np.array(rolloffs)

# 異なる音楽スタイルの特徴量比較
def create_music_styles(sample_rate, duration=4.0):
    """異なる音楽スタイルの信号を生成"""
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    styles = {}
    
    # 1. クラシック風（サイン波ベース、豊富なハーモニー）
    classical = np.zeros_like(t)
    for harmonic in range(1, 6):
        freq = 220 * harmonic
        amplitude = 1.0 / harmonic
        classical += amplitude * np.sin(2 * np.pi * freq * t)
    classical *= np.exp(-t * 0.5)  # 減衰
    styles['クラシック'] = classical
    
    # 2. エレクトロニック（方形波、鋭いエンベロープ）
    electronic = signal.square(2 * np.pi * 440 * t) * 0.5
    # ADSRエンベロープ
    adsr = np.ones_like(t)
    attack_samples = int(0.05 * sample_rate)
    release_start = int(3.5 * sample_rate)
    adsr[:attack_samples] = np.linspace(0, 1, attack_samples)
    adsr[release_start:] = np.linspace(1, 0, len(adsr) - release_start)
    electronic *= adsr
    styles['エレクトロニック'] = electronic
    
    # 3. ロック風（歪み、高エネルギー）
    rock_base = np.sin(2 * np.pi * 220 * t) + 0.5 * np.sin(2 * np.pi * 440 * t)
    # 歪み効果
    rock = np.tanh(rock_base * 3) * 0.7
    styles['ロック'] = rock
    
    # 4. アンビエント（ノイズベース、ローパス）
    np.random.seed(42)
    ambient_noise = np.random.normal(0, 0.3, len(t))
    # ローパスフィルタ
    nyquist = sample_rate / 2
    cutoff = 500 / nyquist
    b, a = signal.butter(4, cutoff, btype='low')
    ambient = signal.filtfilt(b, a, ambient_noise)
    styles['アンビエント'] = ambient
    
    return t, styles

# 特徴量分析
features = MusicFeatures(sample_rate)
t_music, music_styles = create_music_styles(sample_rate)

# 各スタイルの特徴量を計算
style_features = {}
for style_name, audio in music_styles.items():
    style_features[style_name] = {
        'centroid': features.spectral_centroid(audio),
        'bandwidth': features.spectral_bandwidth(audio),
        'zcr': features.zero_crossing_rate(audio),
        'rms': features.rms_energy(audio),
        'rolloff': features.spectral_rolloff(audio)
    }

# 可視化
plt.figure(figsize=(18, 15))

# 各スタイルの波形
for i, (style_name, audio) in enumerate(music_styles.items()):
    plt.subplot(4, 4, i + 1)
    plt.plot(t_music[:4000], audio[:4000], linewidth=1)
    plt.title(f'{style_name}（波形）')
    plt.ylabel('振幅')
    plt.grid(True)

# 特徴量の時間変化
feature_names = ['centroid', 'bandwidth', 'zcr', 'rms']
feature_labels = ['スペクトル重心 (Hz)', '帯域幅 (Hz)', 'ゼロクロッシング率', 'RMSエネルギー']

for i, (feat_name, feat_label) in enumerate(zip(feature_names, feature_labels)):
    plt.subplot(4, 4, i + 5)
    
    for style_name in music_styles.keys():
        feat_values = style_features[style_name][feat_name]
        time_axis = np.linspace(0, len(t_music)/sample_rate, len(feat_values))
        plt.plot(time_axis, feat_values, label=style_name, linewidth=2)
    
    plt.title(feat_label)
    plt.ylabel(feat_label)
    plt.legend()
    plt.grid(True)

# 特徴量の平均値比較
plt.subplot(4, 4, 9)
styles = list(music_styles.keys())
centroid_means = [np.mean(style_features[style]['centroid']) for style in styles]
colors = ['blue', 'red', 'green', 'purple']

bars = plt.bar(styles, centroid_means, color=colors, alpha=0.7)
plt.title('平均スペクトル重心')
plt.ylabel('周波数 (Hz)')
plt.xticks(rotation=45)

for bar, value in zip(bars, centroid_means):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(centroid_means)*0.01,
            f'{value:.0f}', ha='center', va='bottom', fontsize=10)

# ZCR vs RMS散布図
plt.subplot(4, 4, 10)
for i, style in enumerate(styles):
    zcr_mean = np.mean(style_features[style]['zcr'])
    rms_mean = np.mean(style_features[style]['rms'])
    plt.scatter(zcr_mean, rms_mean, color=colors[i], s=100, label=style, alpha=0.8)
    plt.text(zcr_mean, rms_mean, style, fontsize=9, ha='left', va='bottom')

plt.xlabel('平均ゼロクロッシング率')
plt.ylabel('平均RMSエネルギー')
plt.title('ZCR vs RMS')
plt.grid(True)

# 特徴量レーダーチャート用データ準備
plt.subplot(4, 4, 11)
feature_matrix = np.zeros((len(styles), len(feature_names)))

for i, style in enumerate(styles):
    for j, feat_name in enumerate(feature_names):
        feature_matrix[i, j] = np.mean(style_features[style][feat_name])

# 正規化（0-1スケール）
feature_matrix_norm = (feature_matrix - feature_matrix.min(axis=0)) / \
                     (feature_matrix.max(axis=0) - feature_matrix.min(axis=0))

# ヒートマップ
im = plt.imshow(feature_matrix_norm, cmap='viridis', aspect='auto')
plt.xticks(range(len(feature_names)), [name.replace('_', '\n') for name in feature_names])
plt.yticks(range(len(styles)), styles)
plt.title('特徴量ヒートマップ\n（正規化済み）')
plt.colorbar(im, label='正規化値')

# 特徴量統計
plt.subplot(4, 4, 12)
plt.axis('off')

stats_text = "特徴量の特性:\n\n"
for style in styles:
    centroid_mean = np.mean(style_features[style]['centroid'])
    zcr_mean = np.mean(style_features[style]['zcr'])
    stats_text += f"{style}:\n"
    stats_text += f"  重心: {centroid_mean:.0f} Hz\n"
    stats_text += f"  ZCR: {zcr_mean:.3f}\n\n"

plt.text(0.05, 0.95, stats_text, fontsize=10, verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8))

plt.tight_layout()
plt.show()

print("音楽的特徴量の意味:")
print("- スペクトル重心: 音の明るさ（高いほど明るい）")
print("- スペクトル帯域幅: 音色の豊かさ")
print("- ゼロクロッシング率: ノイズ性の指標")
print("- RMSエネルギー: 音の大きさ")
print("- スペクトルロールオフ: 高周波成分の存在")

## 練習問題

1. 実際の音楽ファイルを読み込んで特徴量分析を行ってみましょう
2. 機械学習を使って音楽ジャンル分類システムを構築してみましょう
3. コード進行の自動検出システムを実装してみましょう